In [1]:
# https://thebook.io/080228/part05/ch19/04/

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import UpSampling2D, Flatten, Reshape
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# MNIST 데이터 셋을 호출
(X_train, _), (X_test, _) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float') / 255

In [ ]:
# 생성자 모델 만들기
autoencoder = Sequential()

# 인코딩 부분
autoencoder.add(Conv2D(16, kernel_size=3, padding='same', input_shape=(28,28,1), activation='relu'))
# 출력이 16개, 커널 사이즈=3, 출력이 똑같으므로 패팅=same, input_shape=28,28,1
autoencoder.add(MaxPooling2D(pool_size=2, padding='same'))  # 아래로 가면서 사이즈를 줄여나감
autoencoder.add(Conv2D(8, kernel_size=3, activation='relu', padding='same'))  #.
autoencoder.add(MaxPooling2D(pool_size=2, padding='same'))                    #.
autoencoder.add(Conv2D(8, kernel_size=3, strides=2, padding='same', activation='relu'))  # .

# 디코딩 부분
autoencoder.add(Conv2D(8, kernel_size=3, padding='same', activation='relu'))
autoencoder.add(UpSampling2D())  # 다시 사이즈를 늘려 나감(위에서 줄였으므로)
autoencoder.add(Conv2D(8, kernel_size=3, padding='same', activation='relu'))
autoencoder.add(UpSampling2D())
autoencoder.add(Conv2D(16, kernel_size=3, activation='relu'))
autoencoder.add(UpSampling2D())
autoencoder.add(Conv2D(1, kernel_size=3, padding='same', activation='sigmoid'))  # 이미지 하나

# 전체 구조 확인
autoencoder.summary()

# Output Shape가
# 28,28,16 에서 14,14,16,  7,7,8, ... 로 줄어가다가
# 8,8,8  ,  16,16,8  , ... ,  28,28,1 로 다시 커짐

In [ ]:
# 컴파일 및 학습을 하는 부분
# autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.compile(loss='binary_crossentropy', optimizer='adam')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=128, validation_data=(X_test, X_test)) # 원본을 복제하고 싶은 것

In [ ]:
# 학습된 결과를 출력하는 부분
random_test = np.random.randint(X_test.shape[0], size=5)  
  
# 테스트할 이미지를 랜덤으로 호출
ae_imgs = autoencoder.predict(X_test)  # 앞서 만든 오토인코더 모델에 넣기
 
plt.figure(figsize=(7, 2))  # 출력 이미지 크기 정하기

for i, image_idx in enumerate(random_test):    
   # 랜덤으로 뽑은 이미지를 차례로 나열
   ax = plt.subplot(2, 7, i + 1)
   # 테스트할 이미지를 먼저 그대로 보여줌
   plt.imshow(X_test[image_idx].reshape(28, 28))  
   ax.axis('off')
   ax = plt.subplot(2, 7, 7 + i +1)
   # 오토인코딩 결과를 다음 열에 입력
   plt.imshow(ae_imgs[image_idx].reshape(28, 28))  
   ax.axis('off')
plt.show()

In [ ]:
# 실행 결과에서 
# 첫 번째 줄이 테스트로 사용된 원본 이미지, 
# 두 번째 줄이 원본의 특징을 고스란히 담은 채 만들어진 오토인코더 이미지입니다.